In [1]:
import pandas as pd
import numpy as np
from thefuzz import process

In [3]:
df_listino = pd.read_excel('data_clean/art_costi.xlsx')
df_ddt = pd.read_excel('data_clean/ddt_righe.xlsx')

In [4]:
IDs_listino_univoci = df_listino['Codice_articolo'].unique().tolist()
n_IDs_listino = len(IDs_listino_univoci)
IDs_ddt = df_ddt['Codice_articolo'].tolist()
IDs_ddt_univoci = df_ddt['Codice_articolo'].unique().tolist()
n_IDs_ddt = df_ddt['Codice_articolo'].nunique()

In [10]:
IDs_ddt_not_in_listino = [articolo for articolo in IDs_ddt if articolo not in IDs_listino_univoci]
IDs_ddt_not_in_listino_univoci = list(set(IDs_ddt_not_in_listino))
n_IDs_ddt_not_in_listino = len(IDs_ddt_not_in_listino)
n_IDs_ddt_not_in_listino_univoci = len(IDs_ddt_not_in_listino_univoci)

IDs_listino_not_in_ddt_univoci = [articolo for articolo in IDs_listino_univoci if articolo not in IDs_ddt_univoci]
n_IDs_listino_not_in_ddt = len(IDs_listino_not_in_ddt_univoci)

print('Numero articoli nel listino: ', n_IDs_listino)
print('Numero articoli nei DDT: ', n_IDs_ddt)
print('Numero articoli nei DDT non presenti nel listino: ', n_IDs_ddt_not_in_listino)
print('Numero articoli nei DDT non presenti nel listino (univoci): ', n_IDs_ddt_not_in_listino_univoci)
print('Numero articoli nel listino non presenti nei DDT (univoci): ', n_IDs_listino_not_in_ddt)
print('Percentuale articoli nei DDT non presenti nel listino (univoci): ', n_IDs_ddt_not_in_listino_univoci/n_IDs_ddt)
print('Percentuale articoli nel listino non presenti nei DDT (univoci): ', n_IDs_listino_not_in_ddt/n_IDs_listino)

Numero articoli nel listino:  1311
Numero articoli nei DDT:  1168
Numero articoli nei DDT non presenti nel listino:  12258
Numero articoli nei DDT non presenti nel listino (univoci):  292
Numero articoli nel listino non presenti nei DDT (univoci):  435
Percentuale articoli nei DDT non presenti nel listino:  10.49486301369863
Percentuale articoli nei DDT non presenti nel listino (univoci):  0.25
Percentuale articoli nel listino non presenti nei DDT (univoci):  0.3318077803203661


In [17]:
dict_result = {'ID DDT':[], 'ID più simile nel listino':[], 'score':[]}
for i in range(0, n_IDs_ddt_not_in_listino_univoci):
    dict_result['ID DDT'].append(IDs_ddt_not_in_listino_univoci[i])
    dict_result['ID più simile nel listino'].append(process.extract(IDs_ddt_not_in_listino_univoci[i], IDs_listino_univoci, limit=1)[0][0])
    dict_result['score'].append(process.extract(IDs_ddt_not_in_listino_univoci[i], IDs_listino_univoci, limit=1)[0][1])
df_ddt_simili_listino = pd.DataFrame(dict_result)
df_ddt_simili_listino

,ID DDT,ID più simile nel listino,score
0,017.PM105.04,017.PM105.01,92
1,017.PM117AN,017.PM117,90
2,KCL.MONT,052.ACC.MOB.001,53
3,038.STC44.1F145.01,017.PM103.01,86
4,017.TD.COM.002.985,017.PM001,86
...,...,...,...
287,028.020MDF,028.220,71
288,091.N1046AN,091.N1046,90
289,017.IMB22,017.GUI.MED.01,86
290,090.ANTATR.11,090.VTT6,86


In [18]:
dict_result = {'ID listino':[], 'ID più simile nei DDT':[], 'score':[]}
for i in range(0, n_IDs_listino_not_in_ddt):
    dict_result['ID listino'].append(IDs_listino_not_in_ddt_univoci[i])
    dict_result['ID più simile nei DDT'].append(process.extract(IDs_listino_not_in_ddt_univoci[i], IDs_ddt_univoci, limit=1)[0][0])
    dict_result['score'].append(process.extract(IDs_listino_not_in_ddt_univoci[i], IDs_ddt_univoci, limit=1)[0][1])
df_listino_simili_ddt = pd.DataFrame(dict_result)
df_listino_simili_ddt

,ID listino,ID più simile nei DDT,score
0,017.PM023,017.PM303,89
1,017.PM024,017.PM524,89
2,017.PM025,017.PM056,89
3,017.PM055,017.PM055.H,95
4,017.PM058,017.PM056,89
...,...,...,...
430,092.ANTASCOP.06S,092.ANTASCOP.02S,94
431,092.GU001.10.G,092.GU001.10,95
432,092.TAP01,092.ANTASCTR.02,86
433,098.P055CF6ANT,090.VP015,68


In [20]:
#save both dataframes to csv
df_ddt_simili_listino.to_csv('data_generated/ddt_simili_listino.csv', index=False)
df_listino_simili_ddt.to_csv('data_generated/listino_simili_ddt.csv', index=False)